<A HREF="https://theaidigest.in/topic-modeling-using-roberta-and-transformers/">Topic modeling using Roberta and transformers</A><BR>
    <A HREF="https://maartengr.github.io/BERTopic/index.html">BERTopic documentation</A>

In [3]:
!pip3 install bertopic

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 16.9 MB/s eta 0:00:000:00:010:00:01:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 15.8 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 16.2 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:0031m19.4 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━

  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82814 sha256=5dec19f98fc512c0c8bf7ae2d947f6d8e7c766ebb83ede2b5a36105ce12d30d0
  Stored in directory: /home/muddy/.cache/pip/wheels/fb/99/10/ed2f3bc57ea29f540470eb43570929e30ae911b2d8353b2ee4
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54269 sha256=9df2a05b487604d7bb30d382f5beb9b356ae5ee065bf90a97884c46e563c22e1
  Stored in directory: /home/muddy/.cache/pip/wheels/79/06/ae/9ba8e3fd2ba5e8083c80c3b412e0923e07f4b49ff46522f35d
Successfully built hdbscan sentence-transformers umap-learn pynndescent


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import en_core_web_md
import csv
import regex as re
import os
import spacy
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel
from gensim import corpora
import pprint

import pyLDAvis.gensim_models
pyLDAvis.enable_notebook() #Notebook visualisation enabled

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# Load up the files
path = './DataUCSB/'
list_of_files = []

for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.txt'):
            list_of_files.append(os.path.join(root,file))

filepath = './DataUCSB/address-before-joint-session-the-congress-the-state-the-union-16.csv'
speeches = []
for file in list_of_files:
    with open(file) as f:
        text = f.read()
    speeches.append(text)

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import nltk
from nltk.corpus import stopwords

In [19]:
#Load SpaCy English Model
nlp = en_core_web_md.load()

#Tags to remove
extags = ['PRON','CCONJ','PUNCT','PART','DET','ADP','NUM','SYM','SPACE']

tokens=[]
#SpaCy tokenization + lemmatization + lowercase
for speech in nlp.pipe(speeches):
    scr_tok = [token.lemma_.lower() for token in speech if token.pos_ not in extags and not token.is_stop and token.is_alpha]
    tokens.append(scr_tok)

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
model = BERTopic()

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [43]:
topics = model.fit(speeches)

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [44]:
model.get_topic_info()

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic,Count,Name
0,-1,22,-1_the_of_and_to
1,0,101,0_the_and_to_of
2,1,13,1_the_and_to_of


In [45]:
model.get_topic(0)

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('the', 0.09927994136672218),
 ('and', 0.08518055468397467),
 ('to', 0.08334589426039125),
 ('of', 0.07021807340251841),
 ('that', 0.06051728369716966),
 ('in', 0.052032340633817925),
 ('we', 0.04995927237555015),
 ('our', 0.04363982346407271),
 ('for', 0.03689698029593788),
 ('is', 0.03584071954345856)]

In [46]:
topics = model.transform(speeches)

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [47]:
topics

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


([0,
  0,
  0,
  0,
  0,
  1,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  -1,
  0,
  -1,
  0,
  0,
  0,
  -1,
  0,
  0,
  -1,
  0,
  0,
  1,
  0,
  0,
  0,
  -1,
  -1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  -1,
  0,
  0,
  -1,
  0,
  0,
  0,
  -1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  -1,
  0,
  0,
  0,
  0,
  -1,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  -1,
  0,
  0,
  1,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  0,
  0,
  0,
  0,
  -1,
  -1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  0,
  0,
  -1,
  0,
  0,
  0,
  -1,
  0,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 array([0.93256358, 1.        , 1.        , 0.87156709, 1.        ,
        1.        , 0.        , 0.93186909, 0.98854447, 1.        ,
        1.        , 0.92965651, 0.93186909, 1.        , 0.        ,
        0.        , 1.        , 0.        , 1.        , 1.        ,
        1.        , 0.   

In [48]:
model.get_topics()

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{-1: [('the', 0.1313010769918042),
  ('of', 0.10099139862638366),
  ('and', 0.09753353580760463),
  ('to', 0.08300651030441059),
  ('in', 0.06109972713694986),
  ('that', 0.04520738541707468),
  ('for', 0.04105393947611335),
  ('our', 0.03608205635607054),
  ('is', 0.03429404738549933),
  ('this', 0.03344439781001516)],
 0: [('the', 0.09927994136672218),
  ('and', 0.08518055468397467),
  ('to', 0.08334589426039125),
  ('of', 0.07021807340251841),
  ('that', 0.06051728369716966),
  ('in', 0.052032340633817925),
  ('we', 0.04995927237555015),
  ('our', 0.04363982346407271),
  ('for', 0.03689698029593788),
  ('is', 0.03584071954345856)],
 1: [('the', 0.11391429792155523),
  ('and', 0.08816689959765672),
  ('to', 0.08428017777569825),
  ('of', 0.08344603724229402),
  ('that', 0.07169857839831145),
  ('in', 0.060258254264140484),
  ('we', 0.05324779189000223),
  ('our', 0.03950029486947025),
  ('is', 0.037205143717257375),
  ('this', 0.03681979641443316)]}

In [41]:
model.get_topic(0)

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('he', 0.015122863887999867),
 ('the', 0.013580397938371404),
 ('game', 0.012709322881204655),
 ('year', 0.012178141372182745),
 ('in', 0.01151854037128595),
 ('won', 0.011091010147135294),
 ('games', 0.011067801615291462),
 ('last', 0.01051594879385742),
 ('and', 0.01031874271801776),
 ('phillies', 0.010274980761665053)]

<A HREF="https://maartengr.github.io/BERTopic/getting_started/tips_and_tricks/tips_and_tricks.html#removing-stop-words">Removing stop words</A> a little different than sometimes
<P>
At times, stop words might end up in our topic representations. This is something we typically want to avoid as they contribute little to the interpretation of the topics. However, removing stop words as a preprocessing step is not advised as the transformer-based embedding models that we use need the full context in order to create accurate embeddings.
</P>
<P>
Instead, we can use the CountVectorizer to preprocess our documents after having generated embeddings and clustered our documents. Personally, I have found almost no disadvantages to using the CountVectorizer to remove stopwords and it is something I would strongly advise to try out:
</P>

In [49]:
from sklearn.feature_extraction.text import CountVectorizer

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [50]:
vectorizer_model = CountVectorizer(stop_words="english")
topic_model = BERTopic(vectorizer_model=vectorizer_model)

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [51]:
topics = topic_model.fit(speeches)

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [52]:
topic_model.get_topic(0)

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('people', 0.029917703854087773),
 ('thats', 0.02529476998053627),
 ('just', 0.023321096859245927),
 ('america', 0.022370386784902566),
 ('country', 0.020277043703659303),
 ('american', 0.018564529638358567),
 ('make', 0.01824351458931872),
 ('years', 0.017436705863046405),
 ('new', 0.01702567659659818),
 ('weve', 0.016570754872021455)]

In [54]:
topic_model.get_topics()

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{-1: [('people', 0.028771935959897296),
  ('united', 0.02212140544584723),
  ('states', 0.02179716178665238),
  ('shall', 0.019880274323334328),
  ('thats', 0.018785151760315436),
  ('president', 0.01849457471260058),
  ('order', 0.01643421684209892),
  ('america', 0.01642780783078454),
  ('american', 0.015927392795795876),
  ('going', 0.015587511752194316)],
 0: [('people', 0.029917703854087773),
  ('thats', 0.02529476998053627),
  ('just', 0.023321096859245927),
  ('america', 0.022370386784902566),
  ('country', 0.020277043703659303),
  ('american', 0.018564529638358567),
  ('make', 0.01824351458931872),
  ('years', 0.017436705863046405),
  ('new', 0.01702567659659818),
  ('weve', 0.016570754872021455)],
 1: [('thats', 0.025393323470785413),
  ('health', 0.024674986231095437),
  ('care', 0.02313797756895682),
  ('jobs', 0.023040677005758046),
  ('make', 0.02188971444494992),
  ('people', 0.02165835597839326),
  ('new', 0.021063059444611572),
  ('work', 0.020523252670261376),
  ('insu

In [56]:
topic_model.visualize_topics()

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

/home/muddy/.local/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/home/muddy/.local/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

